In [ ]:
pip install --upgrade --quiet google-cloud-aiplatform[agent_engines,adk]

In [ ]:
pip install -U -q "google-genai"

In [ ]:
pip install --quiet gradio

In [ ]:
#Set environment variables
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = "content-creation-agent-468223"    #Name of Your Project
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"                     #Location
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"                        #Keep it to true to use Vertex AI
os.environ["GOOGLE_CLOUD_BUCKET"] = "gs://launchpad_marketing"          #Bucket used for staging

In [ ]:
#Create Blog post tool

from google import genai
from google.genai.types import Content, CreateCachedContentConfig, HttpOptions, Part
from google.genai import types

def generate_blog_post(
    product_name: str,
    description: str,
    features: str,
    audience:str
):
    
    """
    Generates a product launch blog post using Vertex AI.

    The output includes a title, intro, body, and conclusion — based on the product brief.
    """
    
    model='gemini-2.0-flash'
    client = genai.Client(http_options=HttpOptions(api_version="v1")) 

    
    output_format = """
                    OUTPUT FORMAT (Markdown)

                    # {PRODUCT_NAME}: {Catchy announcement headline}

                    ## Introduction
                    Brief introduction (1 paragraph)

                    ## What’s New
                    Brief paragraph explaining the main improvement or innovation.

                    ## Key Features and Benefits for {TARGET_AUDIENCE}
                    Main body (2–3 paragraphs highlighting benefits/features)

                    ## Conclusion
                    2–3 sentences with a clear call to action.
                    """
    
    # Increase the number of word range so that we allow the model to finish its sentence
    system_instruction = f'''
                            You are Blog Post Agent. 
                            Write a well-structured product announcement blog post that is at least 500 words long and no more than 600 words
                            product-announcement blog post in Markdown.

                            Input fields:
                            - product_name
                            - short_description (1–2 sentences)
                            - key_features (comma-separated)
                            - target_audience (who it’s for)

                            Rules:
                            - Tone: clear, confident, helpful; avoid hype, clichés, and unverifiable claims.
                            - Don’t invent facts. If a required field is missing, ask for it instead of guessing.
                            - Formatting: Markdown only; no images or HTML.
                            - Make sure the blog post does not exceed 600 words and is at least 500 words.

                            {output_format}
                            '''

    user_instruction = f"""
                        Product Name: {product_name}
                        Description: {description}
                        Key Features: {features}
                        Target Audience: {audience}
                        """

    response = client.models.generate_content(
        model=model,
        contents = user_instruction,
        config=types.GenerateContentConfig(
            system_instruction=system_instruction,
            max_output_tokens=900,
            temperature=0.6,
            response_logprobs=True,
            logprobs=3,
          ),
        )

    return response.text

In [ ]:
#Sanity Check for Blog Post Tool

blog_post_response = generate_blog_post(
    product_name = "SmartNotes",
    description = "An AI-powered note-taking app for meetings",
    features = "Real-time transcription, Smart summaries, Action item detection",
    audience = "Remote workers, Product managers, and Sales teams"
)

print(blog_post_response)
print(f"{len(blog_post_response.split())} words")

In [116]:
#Create Agent and set safety settings and hyperparameters

from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp
from google.genai import types

model="gemini-2.0-flash"

#Set safety settings:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
]

generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.7,
   max_output_tokens= 1000,
   top_p= 0.95,
)

#Define Agent
agent = Agent(
    model=model,
    name="Blog_Post_Agent",
    description=("Agent that writes a 400-500 word announcement blog post"),
    generate_content_config=generate_content_config,
    tools=[generate_blog_post],
)

#Create Agent using ADK
app = AdkApp(agent=agent)

In [117]:
#Test Agent
for event in app.stream_query(
    user_id="user_123",
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                """,):
    print(event)

{'content': {'parts': [{'text': 'Okay, I can help you write a blog post announcing VisionLink Pro. I will use the information you provided to generate a draft.\n'}, {'function_call': {'id': 'adk-fc4c545d-2bce-4d5a-85f0-823f95b2763b', 'args': {'description': 'AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration', 'features': 'support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video', 'product_name': 'VisionLink Pro', 'audience': 'remote teams, global companies, and project managers'}, 'name': 'generate_blog_post'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 94, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 94}], 'prompt_token_count': 186, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 186}], 'total_token_count': 280, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-3b29ae17-e851-4626-a1af-3f5d

In [118]:
# Define In-Memory sessions for the Agent
from vertexai.preview.reasoning_engines import AdkApp

def session_service_builder():
  from google.adk.sessions import InMemorySessionService

  return InMemorySessionService()

app = AdkApp(
   agent=agent,                                     
   session_service_builder=session_service_builder,  
)

In [119]:
#Local Sanity Check
#Create Session
session = app.create_session(user_id="user_123")

#Test Agent without giving full information
for event in app.stream_query(
    user_id="user_123",
    session_id=session.id, # Optional. you can pass in the session_id when querying the agent
    message="Create me a blog post about a product called SmartNotes",
):
    print(event)
    #Output should be "Missing target audience, description and features..."

{'content': {'parts': [{'text': 'Okay, I need some more information to write a good blog post.  Tell me:\n\n*   **Who is the audience?** (e.g., Students, Professionals, Researchers)\n*   **What is the product description?** (A brief overview of what SmartNotes does)\n*   **What are the key features?** (List the most important functionalities)\n\nOnce I have this, I can generate a draft for you.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 93, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 93}], 'prompt_token_count': 110, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 110}], 'total_token_count': 203, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-bf9d0a3c-a43a-404a-bd7f-d380530ddcd4', 'author': 'Blog_Post_Agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'c3fe2663-d48c-45c2-97ae-81165fcf1ad6', 'timestamp': 1754696442.353254}


In [120]:
#Give rest of information to Agent
for event in app.stream_query(
    user_id="user_123",
    session_id=session.id,
    message=""" It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                """,):
    print(event)
    #Output: Should now generate the blog post after receiving all of the information

{'content': {'parts': [{'text': 'Okay, I have all the information I need. I will generate a blog post about SmartNotes.\n'}, {'function_call': {'id': 'adk-7b6243c9-af00-47c1-a512-a3443835bc2f', 'args': {'product_name': 'SmartNotes', 'description': 'AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration', 'features': 'Support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video', 'audience': 'Remote teams, global companies, and project managers'}, 'name': 'generate_blog_post'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 86, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 86}], 'prompt_token_count': 276, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 276}], 'total_token_count': 362, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-4487d570-7164-4daa-b7d5-7efc4652a7d7', 'author': 'Blog_Post_Agent', 'act

In [121]:
#Create Agent in Agent Engine (Wait for Creation before moving on...)
import vertexai
from vertexai import agent_engines

vertexai.init(
    project= os.getenv("GOOGLE_CLOUD_PROJECT"),
    location= os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket= os.getenv("GOOGLE_CLOUD_BUCKET"), 
)

remote_agent = agent_engines.create(
    app,
    requirements=["google-cloud-aiplatform[agent_engines,adk]"],  #"cloudpickle==3.1.1","pydantic==2.11.7"
)

Identified the following requirements: {'google-cloud-aiplatform': '1.108.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.7'}
The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]', 'cloudpickle==3.1.1', 'pydantic==2.11.7']
Using bucket launchpad_marketing
Wrote to gs://launchpad_marketing/agent_engine/agent_engine.pkl
Writing to gs://launchpad_marketing/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://launchpad_marketing/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/1014433624001/locations/us-central1/reasoningEngines/5510919404180733952/operations/1046132502910992384
View progress and logs at https://console.cloud.google.com/logs/query?project=content-creation-agent-468223
AgentEngine created. Resource name: projects/1014433

In [122]:
#from vertexai.preview.reasoning_engines import AdkApp
remote_session = remote_agent.create_session(user_id="user_123")

for event in remote_agent.stream_query(
    user_id="user_123",
    session_id = remote_session["id"],
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration. 
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video. 
                The target audience is remote teams, global companies, and project managers."""
   ):
    print(event)

{'content': {'parts': [{'text': 'Okay, I can help you write a blog post draft for the VisionLink Pro announcement. I will use the product name, description, features, and target audience you provided.\n'}, {'function_call': {'id': 'adk-89cd665c-1c2c-4815-8490-84a983089bac', 'args': {'audience': 'remote teams, global companies, and project managers', 'features': 'Support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video', 'product_name': 'VisionLink Pro', 'description': 'AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration'}, 'name': 'generate_blog_post'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 102, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 102}], 'prompt_token_count': 185, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 185}], 'total_token_count': 287, 'traffic_type': 'ON_DEMAND'}, 'invo

In [123]:
#Backend For UI

import os, json, requests
import gradio as gr
from google.auth.transport.requests import Request
import google.auth

# --- Config ---
USER_ID = "user_123"
resource_name = remote_agent.resource_name
location = os.getenv("GOOGLE_CLOUD_LOCATION") or "us-central1"
QUERY_URL  = f"https://{location}-aiplatform.googleapis.com/v1/{resource_name}:query"
STREAM_URL = f"https://{location}-aiplatform.googleapis.com/v1/{resource_name}:streamQuery?alt=sse"

# --- Auth helper ---
def _get_access_token():
    creds, _ = google.auth.default()
    if not creds.valid or creds.expired:
        creds.refresh(Request())
    return creds.token

# --- Session helpers ---
def _extract_session_id(payload: dict) -> str:
    # Your observed shape:
    # { "output": { ..., "id": "8211815291986903040", ... } }
    out = payload.get("output")
    if isinstance(out, dict) and isinstance(out.get("id"), str):
        return out["id"]
    # Fallbacks for other shapes (harmless to keep around)
    if "content" in payload and isinstance(payload["content"], dict):
        sess = payload["content"].get("session")
        if isinstance(sess, dict) and "id" in sess:
            return sess["id"]
    if "session" in payload and isinstance(payload["session"], dict) and "id" in payload["session"]:
        return payload["session"]["id"]
    raise RuntimeError("Unexpected create_session response:\n" + json.dumps(payload, indent=2))

def _create_session():
    token = _get_access_token()
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json; charset=utf-8"}
    body = {"class_method": "create_session", "input": {"user_id": USER_ID}}
    r = requests.post(QUERY_URL, headers=headers, json=body, timeout=60)
    try:
        r.raise_for_status()
    except requests.HTTPError:
        # Return a readable error up the stack
        raise RuntimeError(f"HTTP {r.status_code} creating session:\n{r.text}")
    try:
        payload = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON response from create_session:\n{r.text}")
    return _extract_session_id(payload)

# --- Core call ---
def _stream_query_text(user_input: str, session_id: str) -> str:
    token = _get_access_token()
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json; charset=utf-8",
        # Optional: you can include Accept below; some backends still send JSON lines
        # "Accept": "text/event-stream",
        "Cache-Control": "no-cache",
    }
    body = {
        "class_method": "stream_query",
        "input": {"user_id": USER_ID, "session_id": session_id, "message": user_input}
    }

    result_text = ""
    with requests.post(STREAM_URL, headers=headers, json=body, stream=True, timeout=300) as resp:
        print(f"[agent-debug] stream_query HTTP status: {resp.status_code}")
        print(f"[agent-debug] stream_query headers: {dict(resp.headers)}")
        if not resp.ok:
            return f"[stream_query HTTP {resp.status_code}] {resp.text}"

        for line in resp.iter_lines(decode_unicode=True):
            if not line:
                continue
            print(f"[agent-debug] line: {line}")

            # Support BOTH styles:
            # 1) SSE: "data: {...json...}"
            # 2) NDJSON: "{...json...}"
            payload = line[6:].strip() if line.startswith("data: ") else line.strip()

            try:
                chunk = json.loads(payload)
            except Exception as e:
                print(f"[agent-debug] JSON parse error: {e}; payload head: {payload[:200]}")
                continue

            # Extract text from common envelopes
            content = chunk.get("content") or {}
            parts = content.get("parts", [])
            for p in parts:
                t = p.get("text")
                if isinstance(t, str):
                    result_text += t

            # Also try a couple fallbacks seen in some responses
            if not parts:
                if isinstance(chunk.get("delta"), str):
                    result_text += chunk["delta"]
                out = chunk.get("output")
                if isinstance(out, dict) and isinstance(out.get("text"), str):
                    result_text += out["text"]

    print(f"[agent-debug] Final aggregated reply: {result_text}")
    return result_text.strip()

In [128]:
def query_agent(user_input: str, session_id: str | None):
    try:
        sid = session_id or _create_session()
    except Exception as e:
        # Always return a tuple
        return f"[create_session failed] {e}", session_id

    try:
        reply = _stream_query_text(user_input, sid)
        # Always return (text, session_id)
        return reply, sid
    except Exception as e:
        return f"[stream_query failed] {e}", sid

# --- Gradio wiring (2 outputs expected) ---
def gradio_fn(msg, session_state):
    reply, sid = query_agent(msg, session_state or None)
    return reply, sid


# --- Helper: build the prompt for the Blog Post Agent ---
def compose_prompt(
    product_name, description, features, audience,
    tone, voice, word_count, keywords, cta, sections, region, include_meta
):
    sections_text = ", ".join(sections) if sections else "Announcement, Key features, Benefits, Use cases, CTA"
    guideline = ("Generate a complete first draft with clear headings, short paragraphs, and skimmable bullets where helpful."
    )
    meta_req = "Include an SEO meta title and a meta description at the end." if include_meta else "Do not include meta tags."

    return f"""
Product:
- Name: {product_name or "[Missing]"}
- Description: {description or "[Missing]"}
- Features: {features or "[Missing]"}
- Target Audience: {audience or "[Missing]"}

Writing Requirements:
- Tone: {tone}
- Voice: {voice}
- Target Length: ~{word_count} words (flexible)
- SEO Keywords: {keywords or "none"}
- Call To Action: {cta or "none"}
- Region/Locale considerations: {region or "global"}
- Sections to include: {sections_text}
"""

# --- Helper to calculate word count
def _add_word_count(text: str) -> str:
    words = len(text.split())
    return f"**Word Count:** {words}\n\n{text}"

def ensure_prompt_or_compose(prompt_text,
    product_name, description, features, audience,
    tone, voice, word_count, keywords, cta, sections, region, include_meta
):
    # If user provided a manual prompt, use it verbatim
    if prompt_text and prompt_text.strip():
        return prompt_text.strip()
    # Otherwise compose from fields
    return compose_prompt(
        product_name, description, features, audience,
        tone, voice, word_count, keywords, cta,
        sections, region, include_meta,
    )
    
def build_prompt_from_fields(
    product_name, description, features, audience,
    tone, voice, word_count, keywords, cta,
    sections, region, include_meta
):
    prompt = compose_prompt(
        product_name, description, features, audience,
        tone, voice, word_count, keywords, cta,
        sections, region, include_meta,
    )
    return prompt

def send_full(
    prompt_text,
    product_name, description, features, audience,
    tone, voice, word_count, keywords, cta,
    sections, region, include_meta, session_state
):
    prompt = ensure_prompt_or_compose(
        prompt_text,
        product_name, description, features, audience,
        tone, voice, word_count, keywords, cta,
        sections, region, include_meta,
    )
    reply, sid = query_agent(prompt, session_state or None)
    return prompt, f"**Word Count:** {len(reply.split())}\n\n{reply}", sid

# --- Sample product prompts (fills the fields) ---
sample_prompts = [
    {
        "label": "📱 Smartwatch Launch",
        "values": {
            "product_name": "PulseSync One",
            "description": "A sleek smartwatch with multi-day battery, GPS, heart-rate & sleep tracking, and seamless phone integration.",
            "features": "7-day battery, AMOLED display, GPS, Heart-rate, SpO2, Sleep stages, 5ATM water resistance, Fast charge",
            "audience": "Health-conscious professionals and casual athletes",
            "tone": "Informative",
            "voice": "Brand-forward",
            "word_count": 900,
            "keywords": "smartwatch, fitness tracking, long battery life",
            "cta": "Pre-order PulseSync One today",
            "sections": ["Intro", "What’s new", "Key features", "Real-world benefits", "Pricing & availability", "CTA"],
            "region": "North America",
            "include_meta": True
        }
    },
    {
        "label": "🎧 Wireless Earbuds",
        "values": {
            "product_name": "SoundSphere Pro",
            "description": "Premium Bluetooth earbuds with adaptive noise canceling and crystal-clear calls.",
            "features": "ANC, Transparency mode, Multipoint pairing, Wireless charging, IPX5",
            "audience": "Commuters and remote workers",
            "tone": "Conversational",
            "voice": "Customer-first",
            "word_count": 800,
            "keywords": "wireless earbuds, ANC, Bluetooth 5.3",
            "cta": "Shop SoundSphere Pro",
            "sections": ["Intro", "What’s new", "Key features", "Real-world benefits", "Pricing & availability", "CTA"],
            "region": "Global",
            "include_meta": True
        }
    },
    {
        "label": "☕ Smart Coffee Maker",
        "values": {
            "product_name": "BrewMaster 3000",
            "description": "Wi-Fi coffee maker with app scheduling, brew-strength control, and built-in grinder.",
            "features": "App control, Programmable schedules, Built-in grinder, Energy saver, Descale reminders",
            "audience": "Busy professionals and coffee lovers",
            "tone": "Warm",
            "voice": "Helpful expert",
            "word_count": 700,
            "keywords": "smart coffee maker, app controlled, home brewing",
            "cta": "Get BrewMaster 3000",
            "sections": ["Intro", "What’s new", "Key features", "Real-world benefits"],
            "region": "US & Canada",
            "include_meta": False
        }
    },
]

# --- UI ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ✍️ Eddy Daouk's Blog Post Agent — Product Announcement Drafts\nGive the agent your product details and generate a full first draft.")

    state = gr.State(value=None)  # session_id

    with gr.Row():
        with gr.Column(scale=5):
            with gr.Row():
                product_name = gr.Textbox(label="Product Name", placeholder="e.g., PulseSync One")
                region = gr.Textbox(label="Region/Locale", value="Global")

            description = gr.Textbox(label="Short Product Description", lines=3, placeholder="1–3 sentences about the product and what makes it notable.")
            features = gr.Textbox(label="Key Features (comma-separated)", placeholder="e.g., ANC, Wireless Charging, IPX5, Fast Pair")
            audience = gr.Textbox(label="Target Audience", placeholder="e.g., Commuters, Remote Workers, Fitness Enthusiasts")

            with gr.Row():
                tone = gr.Dropdown(choices=["Informative", "Conversational", "Warm", "Playful", "Professional"], value="Informative", label="Tone")
                voice = gr.Dropdown(choices=["Brand-forward", "Customer-first", "Helpful expert", "Neutral"], value="Brand-forward", label="Voice")
                word_count = gr.Slider(400, 1500, value=900, step=50, label="Target Words")

            keywords = gr.Textbox(label="SEO Keywords (comma-separated)", placeholder="e.g., smartwatch, fitness tracking, long battery life")
            cta = gr.Textbox(label="Call To Action", placeholder="e.g., Pre-order today")

            sections = gr.CheckboxGroup(
                choices=["Intro", "Announcement", "What’s new", "Key features", "Real-world benefits", "Setup", "Pricing & availability", "CTA"],
                value=["Intro", "What’s new", "Key features", "Real-world benefits", "CTA"],
                label="Sections to Include"
            )
            include_meta = gr.Checkbox(label="Include SEO meta title & description", value=True)
                

        with gr.Column(scale=5):
            gr.Markdown("### Prompt Preview")
            prompt_editor = gr.Textbox(label="Prompt", lines=14, placeholder="Click 'Build Prompt' to generate, then edit as needed.")
            with gr.Row():
                build_prompt_btn = gr.Button("🧩 Build Prompt from Fields")
                btn_full = gr.Button("📝 Generate Full Draft", variant="primary")
            gr.Markdown("### Output")
            chat_out = gr.Markdown(value="*(Your draft will appear here)*")
            gr.Markdown("### Session")
            session_view = gr.Textbox(label="Session ID", interactive=False)
            gr.Markdown("### Quick Examples")
            example_btns = []
            for ex in sample_prompts:
                example_btns.append(gr.Button(ex["label"], size="sm"))

    # --- Hook up buttons ---
    build_prompt_btn.click(
    build_prompt_from_fields,
    inputs=[product_name, description, features, audience, tone, voice, word_count, keywords, cta, sections, region, include_meta],
    outputs=[prompt_editor]
    )

    btn_full.click(
        send_full,
        inputs=[prompt_editor, product_name, description, features, audience, tone, voice, word_count, keywords, cta, sections, region, include_meta, state],
        outputs=[prompt_editor, chat_out, session_view]
    )

    # Fill fields from examples
    def load_example(vals):
        return (
            vals["product_name"], vals["description"], vals["features"], vals["audience"],
            vals["tone"], vals["voice"], vals["word_count"], vals["keywords"], vals["cta"],
            vals["sections"], vals["region"], vals["include_meta"]
        )

    for i, ex in enumerate(sample_prompts):
        example_btns[i].click(
            lambda v=ex["values"]: load_example(v),
            inputs=None,
            outputs=[product_name, description, features, audience, tone, voice, word_count, keywords, cta, sections, region, include_meta]
        )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://383c45c4e08a910a2b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[agent-debug] stream_query HTTP status: 200
[agent-debug] stream_query headers: {'Content-Type': 'application/json', 'Content-Disposition': 'attachment', 'Vary': 'Origin, X-Origin, Referer', 'Transfer-Encoding': 'chunked', 'Date': 'Sat, 09 Aug 2025 00:10:01 GMT', 'Server': 'scaffolding on HTTPServer2', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff'}
[agent-debug] line: {"content": {"parts": [{"text": "Okay, I can help you write a blog post draft for the PulseSync One smartwatch. I will use the `generate_blog_post` function with the information you provided.\n\n"}, {"function_call": {"id": "adk-bd128ee1-0199-4c50-87f3-8897e7a809d7", "args": {"description": "A sleek smartwatch with multi-day battery, GPS, heart-rate & sleep tracking, and seamless phone integration.", "audience": "Health-conscious professionals and casual athletes", "features": "7-day battery, AMOLED display, GPS, Heart-rate, SpO2, Sleep stages, 5ATM water resistance, Fast ch

In [127]:
#Cleanup
demo.close()

Closing server running on port: 7864


In [104]:
remote_agent.delete(force=True)

Delete Agent Engine backing LRO: projects/1014433624001/locations/us-central1/operations/7970557667481485312
Agent Engine deleted. Resource name: projects/1014433624001/locations/us-central1/reasoningEngines/5075196140232638464
